In [1]:
import pandas as pd
from datatable import dt, f

# Aquisição dos dados brutos

como os dados foram adquiridos (decrição de atualizações na base e etc)


## Áreas STEM

- 51: ciências biológicas e correlatas

- 54: matemática e estatística

- 53: ciências físicas

- 61: computação

- 71: engenharias e correlatas

In [2]:
def filter_dataset(dataset):
  df = dataset[
    (
      (f.area == 51)
      |
      (f.area == 54)
      |
      (f.area == 53)
      |
      (f.area == 61)
      |
      (f.area == 61)
      |
      (f.area == 71)
    ) & (
      (f.ingressantes > 0)
      &
      (f.concluintes > 0)
    )
    , :]
  return df

In [3]:
def get_path_table_by_year(year):
  path_table = '/'.join([
    '../data/raw',
    'censo_{year}.zip',
    'Microdados do Censo da Educaç╞o Superior {year}',
    'dados',
    'MICRODADOS_CADASTRO_CURSOS_{year}.CSV'
  ]).format(year = year)
  return path_table

In [4]:
def read_datasets(from_year, to_year, columns):
  dfs = list()
  for y in range(from_year, to_year + 1):
    path_table = get_path_table_by_year(y)
    dataset = dt.fread(path_table, columns=columns)
    df_filter = filter_dataset(dataset)
    dfs.append(df_filter)
  df =  dt.rbind(*dfs)
  df = df.sort(f.ano, f.cod_municipio, f.area, f.curso, -f.concluintes_fem)
  df = df.to_pandas().dropna()
  return df

## Colunas

- NU_ANO_CENSO: Ano do Censo

- NO_REGIAO: Nome da região geográfica do local de oferta do curso

- SG_UF: Sigla da Unidade da Federação do local de oferta do curso

- NO_MUNICIPIO: Nome do Município do local de oferta do curso

- CO_MUNICIPIO: Código do IBGE do municipio do local de oferta do curso

- IN_CAPITAL: Informa se o local de oferta do curso está localizado em capital de Estado
	- 0. Não
	- 1. Sim
	- (.) Não aplicável (Curso a distância)

- TP_REDE: Rede de Ensino
	- 1. Pública
	- 2. Privada

- CO_CINE_AREA_ESPECIFICA: Nome da área específica, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco

- NO_CINE_ROTULO: Nome do curso, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco

- TP_GRAU_ACADEMICO: Tipo do grau acadêmico conferido ao ao aluno pela conclusão dos requisitos exigidos pelo curso
	- 1. Bacharelado
	- 2. Licenciatura
	- 3. Tecnológico
	- 4. Bacharelado e Licenciatura
	- (.) Não aplicável (cursos com nivel acadêmico igual a sequencial de formação específica ou  cursos de área básica de curso identificada pela variável TP_ATRIBUTO_INGRESSO)

- TP_MODALIDADE_ENSINO: Tipo da modalidade de ensino do curso
	- 1. Presencial
	- 2. Curso a distância

- TP_NIVEL_ACADEMICO: Tipo do nível acadêmico do curso
	- 1. Graduação
	- 2. Seqüencial de Formação Específica

- QT_ING: Quantidade de ingressante no curso

- QT_ING_FEM: Quantidade de ingressante do sexo feminino no curso

- QT_CONC: Quantidade de concluinte no curso

- QT_CONC_FEM: Quantidade de concluinte no curso do sexo feminino

In [5]:
columns = {
  'NU_ANO_CENSO': 'ano', #  Ano do Censo
  'NO_REGIAO': 'regiao', #  Nome da região geográfica do local de oferta do curso
  'SG_UF': 'uf', #  Sigla da Unidade da Federação do local de oferta do curso
  'CO_MUNICIPIO': 'cod_municipio', #  Código do IBGE do municipio do local de oferta do curso
  'NO_MUNICIPIO': 'nome_municipio', #  Nome do Município do local de oferta do curso
  'IN_CAPITAL': 'capital', #  Informa se o local de oferta do curso está localizado em capital de Estado
  'CO_CINE_AREA_ESPECIFICA': 'area', #  Nome da área específica, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco
  'NO_CINE_ROTULO': 'curso', #  Nome do curso, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco
  'TP_REDE': 'rede_ensino', #  Rede de Ensino
  'TP_MODALIDADE_ENSINO': 'modalidade', #  Tipo da modalidade de ensino do curso
  'TP_GRAU_ACADEMICO': 'grau_academico', #  Tipo do grau acadêmico conferido ao ao aluno pela conclusão dos requisitos exigidos pelo curso
  'TP_NIVEL_ACADEMICO': 'nivel_academico', #  Tipo do nível acadêmico do curso
  'QT_ING': 'ingressantes', #  Quantidade de ingressante no curso
  'QT_ING_FEM': 'ingressantes_fem', #  Quantidade de ingressante do sexo feminino no curso
  'QT_CONC': 'concluintes', #  Quantidade de concluinte no curso
  'QT_CONC_FEM': 'concluintes_fem', #  Quantidade de concluinte no curso do sexo feminino
  ...: None,
}

In [6]:
from_year = 2010
to_year = 2019
df = read_datasets(from_year, to_year, columns)
df = df[list(columns.values())[:-1]]
df

,ano,regiao,uf,cod_municipio,nome_municipio,capital,area,curso,rede_ensino,modalidade,grau_academico,nivel_academico,ingressantes,ingressantes_fem,concluintes,concluintes_fem
0,2010,Norte,RO,1100023.0,Ariquemes,False,61,Sistemas de informação,2,1,1.0,1,2,0,15,1
1,2010,Norte,RO,1100049.0,Cacoal,False,51,Biologia,2,1,1.0,1,21,13,6,4
2,2010,Norte,RO,1100049.0,Cacoal,False,61,Sistemas de informação,2,1,3.0,1,20,4,25,5
3,2010,Norte,RO,1100049.0,Cacoal,False,71,Gestão ambiental,2,1,3.0,1,61,26,27,12
4,2010,Norte,RO,1100049.0,Cacoal,False,71,Gestão ambiental,2,2,3.0,1,88,48,10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48779,2019,Centro-Oeste,DF,5300108.0,Brasília,True,71,Gestão ambiental,2,2,3.0,1,10,4,3,1
48780,2019,Centro-Oeste,DF,5300108.0,Brasília,True,71,Gestão ambiental,2,2,3.0,1,10,2,1,1
48781,2019,Centro-Oeste,DF,5300108.0,Brasília,True,71,Gestão ambiental,2,2,3.0,1,4,3,2,1
48782,2019,Centro-Oeste,DF,5300108.0,Brasília,True,71,Gestão ambiental,2,2,3.0,1,4,0,1,1


In [7]:
output_file = '../data/cursos.csv.zip'
df.to_csv(output_file, index=False)

In [8]:
df2 = pd.read_csv(output_file, dtype={'cod_municipio': int, 'grau_academico':int})
df2

,ano,regiao,uf,cod_municipio,nome_municipio,capital,area,curso,rede_ensino,modalidade,grau_academico,nivel_academico,ingressantes,ingressantes_fem,concluintes,concluintes_fem
0,2010,Norte,RO,1100023,Ariquemes,False,61,Sistemas de informação,2,1,1,1,2,0,15,1
1,2010,Norte,RO,1100049,Cacoal,False,51,Biologia,2,1,1,1,21,13,6,4
2,2010,Norte,RO,1100049,Cacoal,False,61,Sistemas de informação,2,1,3,1,20,4,25,5
3,2010,Norte,RO,1100049,Cacoal,False,71,Gestão ambiental,2,1,3,1,61,26,27,12
4,2010,Norte,RO,1100049,Cacoal,False,71,Gestão ambiental,2,2,3,1,88,48,10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48712,2019,Centro-Oeste,DF,5300108,Brasília,True,71,Gestão ambiental,2,2,3,1,10,4,3,1
48713,2019,Centro-Oeste,DF,5300108,Brasília,True,71,Gestão ambiental,2,2,3,1,10,2,1,1
48714,2019,Centro-Oeste,DF,5300108,Brasília,True,71,Gestão ambiental,2,2,3,1,4,3,2,1
48715,2019,Centro-Oeste,DF,5300108,Brasília,True,71,Gestão ambiental,2,2,3,1,4,0,1,1
